In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
g_key = "(input your key here)"

##  Store Part I results into DataFrame

In [ ]:
# Load the csv exported in Part I to a DataFrame
weather_df = pd.read_csv('../OpenWeatherPy/output_data/cities.csv')
weather_df.head()

## Humidity Heatmap

In [ ]:
# Configure gmaps
gmaps.configure(api_key = g_key)

In [ ]:
# Use the Lat and Lng as locations and Humidity as the weight
locations = weather_df[["Lat", "Lng"]]
humidity = weather_df["Humidity"]

In [ ]:
#Add Heatmap layer to map
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
max_intensity = np.max(humidity)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

## Create new DataFrame fitting weather criteria

In [ ]:
# Narrow down the cities to fit weather conditions on 3 conditions: A max temperature lower than 80 degrees but higher than 70, 
    # Wind speed less than 10 mph and zero cloudiness.
weather_criteria = weather_df.loc[(weather_df["Max Temp"] > 70) & (weather_df["Max Temp"] < 80) & 
                                  (weather_df["Wind Speed"] <= 10) & (weather_df["Cloudiness"] == 0), :]

# Drop any rows with null values
weather_criteria = weather_criteria.dropna(how='any')

# Reset index 
weather_criteria.reset_index(inplace=True)
del weather_criteria['index']

# Print 
weather_criteria.head()

## Hotel Map

In [ ]:
# Store into variable named hotel_df.
hotel_df = weather_criteria.loc[:,["City","Country", "Lat", "Lng"]]

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""
hotel_df

In [ ]:
import time

# Set parameters to search for hotels with 5000 meters.
params = {
        "location": city_coords,
        "radius": 5000,
        "types" : "lodging",
        "key": g_key
}
# Hit the Google Places API for each city's coordinates.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Store the first Hotel result into the DataFrame.
hotels = []

for index, row in hotel_df.iterrows():

    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    params["location"] = f"{lat},{lng}"
    
    hotel_request = requests.get(base_url, params=params)
    hotel_response = hotel_request.json()
    
    try:
        hotels.append(hotel_response["results"][0]["name"])
    except:
        hotels.append("Nearest hotel not found")
        
    time.sleep(1)
        
weather_criteria["Hotel Name"] = hotels
weather_criteria = weather_criteria.dropna(how='any')
weather_criteria.head()

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content= [f"Nearest Hotel: {hotel}" for hotel in hotels])
fig.add_layer(markers)

# Display figure
fig